## Web Scraping
*** Go to "Rate Your Music" website and retrieve a chart of the top albums of all time ***

In [14]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
import pandas as pd


In [2]:
from webdriver_manager.chrome import ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\bentl\.wdm\drivers\chromedriver\win32\88.0.4324.96]


In [3]:
url = 'https://rateyourmusic.com/charts/top/album/all-time/'
browser.visit(url)

In [56]:
# Iterate through all pages
for x in range(1):
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain album information
    albums = soup.find_all('div', class_='topcharts_itembox chart_item_release')

    album_list = []
    # Iterate through each album
    for album in albums:
        # Use Beautiful Soup's find() method to navigate and retrieve album attributes
        try:
            album_dict = {
                "position" : album.find('div', class_='topcharts_position').text,
                "title" : album.find('a', class_='release').text,
                "artist" : album.find('a', class_='artist').text,
                "release_date" : album.find('div', class_='topcharts_item_releasedate').text.strip('\t\r\n'),
                "genres" : album.find('div', class_='topcharts_item_genres_container').text.strip('\t\r\n'),
                "secondarygenres" : album.find('div', class_='topcharts_item_secondarygenres_container').text.strip('\t\r\n'),
                "avg_rating" : album.find('span', class_='topcharts_stat topcharts_avg_rating_stat').text,
                "rating_count" : album.find('span', class_='topcharts_stat topcharts_ratings_stat').text,
                "review_count" : album.find('span', class_='topcharts_stat topcharts_reviews_stat').text,
                "spotify_link" : album.find('a', class_='ui_media_link_btn ui_media_link_btn_spotify').attrs.get("href") }
               
        except AttributeError:
            print('no record')
        
        album_list.append(album_dict)

    # Click the 'Next' button on each page
    try:
        browser.click_link_by_partial_text('Next')
          
    except:
        print("Scraping Complete")

    time.sleep(1)

top_albums = pd.DataFrame(album_list)


no record
no record


In [57]:
top_albums.head(10)

,position,title,artist,release_date,genres,secondarygenres,avg_rating,rating_count,review_count,spotify_link
0,561.,Twin Fantasy,Car Seat Headrest,16 February 2018 \n \n ...,Indie Rock,"Singer/Songwriter, \nPower Pop",3.85,"15,384",200,https://open.spotify.com/album/20U1UWeGcGq7JVW...
1,562.,A Crow Looked at Me,Mount Eerie,24 March 2017 \n \n ...,"Singer/Songwriter, \nIndie Folk, \nContemporar...",Slowcore,3.86,"12,565",190,https://open.spotify.com/album/5p64XgvFREt1P6m...
2,563.,The Fragile,Nine Inch Nails,21 September 1999 \n \n ...,"Industrial Rock, \nArt Rock, \nAlternative Rock","Industrial Metal, \nAmbient",3.89,"10,795",221,https://open.spotify.com/album/4uiVwLbTzE6VMkX...
3,564.,Scott 4,Scott Engel,November 1969 \n \n ...,"Baroque Pop, \nSinger/Songwriter",,3.91,"7,506",134,https://open.spotify.com/album/7uU2qrFZQSdQaci...
4,565.,Elvis: TV Special,Elvis Presley,December 1968 \n Live,"Rock & Roll, \nPop Rock","Blue-Eyed Soul, \nGospel, \nRockabilly, \nRhyt...",3.98,595,28,https://open.spotify.com/album/4OaxrDxZe97lotU...
5,566.,Deep Purple in Concert,Deep Purple,December 1980 \n Live,Hard Rock,Progressive Rock,3.99,548,23,https://open.spotify.com/album/63KHr72RpRS3k0A...
6,567.,Ella in Berlin: Mack the Knife,Ella Fitzgerald,1960\n Live,"Vocal Jazz, \nStandards",Swing,3.98,487,19,https://open.spotify.com/album/1wQ8jRyqbHquuUB...
7,568.,Live in Hamburg,Esbjörn Svensson Trio,23 November 2007 \n ...,"ECM Style Jazz, \nAvant-Garde Jazz","Post-Bop, \nJazz Fusion",4.00,281,10,https://open.spotify.com/album/3PU1b3aDrbMKucR...
8,569.,Individual Thought Patterns,Death,30 June 1993 \n \n ...,Technical Death Metal,Progressive Metal,3.90,"7,825",132,https://open.spotify.com/album/56gbykNskCFungP...
9,570.,Matthäus-Passion,Collegium Vocale Gent,16 November 1999 \n \n ...,"Baroque Music, \nOratorio, \nChristian Liturgi...",Choral,3.96,741,7,https://open.spotify.com/album/5ZHCqltda5ywYkH...


*** Transform ***
* Remove end of line characters
* set the index to the position

In [58]:
top_albums['release_date'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)
top_albums['genres'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)
top_albums['secondarygenres'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)
top_albums['position'] = top_albums['position'].str.replace('[^\w\s]','')


In [59]:
top_albums.head(15)

,position,title,artist,release_date,genres,secondarygenres,avg_rating,rating_count,review_count,spotify_link
0,561,Twin Fantasy,Car Seat Headrest,16 February 2018,Indie Rock,"Singer/Songwriter, Power Pop",3.85,"15,384",200,https://open.spotify.com/album/20U1UWeGcGq7JVW...
1,562,A Crow Looked at Me,Mount Eerie,24 March 2017,"Singer/Songwriter, Indie Folk, Contemporary Folk",Slowcore,3.86,"12,565",190,https://open.spotify.com/album/5p64XgvFREt1P6m...
2,563,The Fragile,Nine Inch Nails,21 September 1999,"Industrial Rock, Art Rock, Alternative Rock","Industrial Metal, Ambient",3.89,"10,795",221,https://open.spotify.com/album/4uiVwLbTzE6VMkX...
3,564,Scott 4,Scott Engel,November 1969,"Baroque Pop, Singer/Songwriter",,3.91,"7,506",134,https://open.spotify.com/album/7uU2qrFZQSdQaci...
4,565,Elvis: TV Special,Elvis Presley,December 1968 Live,"Rock & Roll, Pop Rock","Blue-Eyed Soul, Gospel, Rockabilly, Rhythm & B...",3.98,595,28,https://open.spotify.com/album/4OaxrDxZe97lotU...
5,566,Deep Purple in Concert,Deep Purple,December 1980 Live,Hard Rock,Progressive Rock,3.99,548,23,https://open.spotify.com/album/63KHr72RpRS3k0A...
6,567,Ella in Berlin: Mack the Knife,Ella Fitzgerald,1960 Live,"Vocal Jazz, Standards",Swing,3.98,487,19,https://open.spotify.com/album/1wQ8jRyqbHquuUB...
7,568,Live in Hamburg,Esbjörn Svensson Trio,23 November 2007 Live,"ECM Style Jazz, Avant-Garde Jazz","Post-Bop, Jazz Fusion",4.00,281,10,https://open.spotify.com/album/3PU1b3aDrbMKucR...
8,569,Individual Thought Patterns,Death,30 June 1993,Technical Death Metal,Progressive Metal,3.90,"7,825",132,https://open.spotify.com/album/56gbykNskCFungP...
9,570,Matthäus-Passion,Collegium Vocale Gent,16 November 1999,"Baroque Music, Oratorio, Christian Liturgical ...",Choral,3.96,741,7,https://open.spotify.com/album/5ZHCqltda5ywYkH...
